# Analyse des ressentis avec GloVe Vectors

L'analyse des ressentis a pour objectif de classifier les opinions exprimées sous forme de texte dans un langage naturel. Différentes méthodes peuvent être utlisées pour réaliser cette analyse. Dans cet exemple, des commentaires recueillis surle site [allociné](http://www.allocine.fr/) vont être utlisés pour entrainer un modèle de classification supervisé. Le modèle utilise un réseau de neurones à convolution 1D.

Les mots contenus dans les phrases doivent être encodés dans des vecteurs qui seront utilisés pour l'entrainement et les tests. Une manière simple serait d'assigner à chaque mot une valeur numérique. Le problème de cette méthode est que le contexte dans lequel les mots sont utilisés n'est pas pris en compte. Une méthode permettant de combler ce manque est d'utiliser un algorithme de prolongation lexicale (Word embedding). Parmi ce genre d'algorithme, on trouve [GloVe (Global Vectors for Word Representation)](https://nlp.stanford.edu/projects/glove/) qui se base sur les probabilités de co-occurence de mots. Une base de données pré-entrainée (mais en Anglais) est déjà disponnible sur leur site web.  

Une base de donnée pré-entrainée en Français peut-être néanmoins téléchargée à cette [adresse](http://www.cs.cmu.edu/~afm/projects/multilingual_embeddings.html). C'est celle-ci que nous utiliserons.

In [14]:
import pandas as pd
import numpy as np
import random
import json

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# Chargement des données d'entrainement

Les données d'entrainement sont chargées dans une dataframe Panda depuis des fichiers textes. Ces données contiennent deux colonnes : Un commentaire et une valeur binaire définissant un sentiment négatif ou positif sur ce commentaire. 

Ces données sont disponnibles sur le github de [TheophileBlard](https://github.com/TheophileBlard/french-sentiment-analysis-with-bert/tree/master/allocine_dataset). Ces données sont issues de commentaires récupérés sur Allociné. Les utilisateurs votent avec des notes allant de 0.5 à 5.0 :    
<img src="https://github.com/AlexandreBourrieau/ML-F1/blob/master/Carnets%20Jupyter/Images/rating_counts.png?raw=1" width="600"/>

Afin de récupérer une note binaire (négative ou positive) à partir de cet intervalle de valeurs, les votes <= 2 sont classé comme négatifs et ceux >=4 sont classés comme positifs. Les autres sont classés comme neutres :    
<img src="https://github.com/AlexandreBourrieau/ML-F1/blob/master/Carnets%20Jupyter/Images/polarity_frequency.png?raw=1" width="600"/>  
  

Enfin, pour construire les données, 100 000 avis négatifs et 100 000 avis positifs sont extraits aléatoirement, puis décomposés en deux catégories : Les données d'entrainement (80%), les données de test (10%) et les données de validation (10%) :  
<img src="https://github.com/AlexandreBourrieau/ML-F1/blob/master/Carnets%20Jupyter/Images/splits_polarity.png?raw=1" width="600"/>  



In [ ]:
# Téléchargement des données depuis le repot github "https://github.com/AlexandreBourrieau/ML-F1/raw/master/Carnets%20Jupyter/Donn%C3%A9es/data.tar.bz2"

!wget "https://github.com/AlexandreBourrieau/ML-F1/raw/master/Carnets%20Jupyter/Donn%C3%A9es/data.tar.bz2"
!tar -xjvf data.tar.bz2
!ls -l data

In [15]:
DataEntrainement = pd.read_json("/content/data/test.jsonl", lines=True)
DataEntrainement.head(10)

,film-url,review,polarity
0,http://www.allocine.fr/film/fichefilm-25385/cr...,"Magnifique épopée, une belle histoire, touchan...",1
1,http://www.allocine.fr/film/fichefilm-1954/cri...,Je n'ai pas aimé mais pourtant je lui mets 2 é...,0
2,http://www.allocine.fr/film/fichefilm-135523/c...,Un dessin animé qui brille par sa féerie et se...,1
3,http://www.allocine.fr/film/fichefilm-61514/cr...,"Si c'est là le renouveau du cinéma français, c...",0
4,http://www.allocine.fr/film/fichefilm-260395/c...,Et pourtant on s’en Doutait !Second volet très...,0
5,http://www.allocine.fr/film/fichefilm-220641/c...,Vous reprendrez bien un peu d'été ? Ce film je...,1
6,http://www.allocine.fr/film/fichefilm-120103/c...,Bon c'est pas un grand film mais on passe un b...,1
7,http://www.allocine.fr/film/fichefilm-190956/c...,"Terrible histoire que ces êtres sans amour, ce...",1
8,http://www.allocine.fr/film/fichefilm-186185/c...,"Un très joli film, qui ressemble à un téléfilm...",1
9,http://www.allocine.fr/film/fichefilm-17327/cr...,Mais comment certaines personnes ont pus lui m...,0


# Chargement des vecteurs GloVe et préparation des données

**Commençons par charger les vecteurs GloVe**  

Les fichiers des vecteurs GloVe sont téléchargés à partir du site de [GloVe](https://nlp.stanford.edu/projects/glove/) mais ils sont en Anglais. Nous allons utiliser une [version française](http://www.cs.cmu.edu/~afm/projects/multilingual_embeddings.html) équivalente.  
  
Le fichier que nous utilisons contient plus de 40000 vecteurs de dimension 300. Cela signifie que pour chaque mot contenu dans ce fichier, un vecteur de 300 valeurs permet de définir les relations lexicales de ce mots avec les autres mots du fichier.  

<img src="https://github.com/AlexandreBourrieau/ML-F1/blob/master/Carnets%20Jupyter/Images/EmbeddedVectors.png?raw=1"/>  

In [ ]:
# Téléchargement des vecteurs
!wget "https://github.com/AlexandreBourrieau/ML-F1/raw/master/Carnets%20Jupyter/Donn%C3%A9es/multilingual_embeddings.rar"
!mv multilingual_embeddings.rar /content/data/multilingual_embeddings.rar
!unrar x /content/data/multilingual_embeddings.rar /content/data

**Encodage des commentaires**  
Pour utiliser notre réseau de neurones, nous devons encoder le texte des commentaires. L'encodage suit le principe suivant :  
* Chaque mot contenu dans l'ensemble des commentaires va se voir attribuer une valeur entière unique
* Chaque commentaire va ensuite être transformé en un vecteur de nombre entiers, dont les nombres correspondent aux valeurs entières attribuées aux mots précédemment  
* Les commentaires sont ensuite redimensionnés afin d'avoir tous la même dimension (avec bourrage de 0)

In [ ]:
print(DataEntrainement['review'][0])
print(DataEntrainement['review'][1])

Voici un exemple d'encodage de texte :

In [ ]:
com = ['un plus un egal deux','deux plus deux cela fait quatre']
tokenizer_ex = Tokenizer(num_words=10)
tokenizer_ex.fit_on_texts(com)
seq = tokenizer_ex.texts_to_sequences(com)
bourrage = pad_sequences(seq,maxlen=10)

print(com)
print(tokenizer_ex.word_index)
print(seq)
print(bourrage)

Le code suivant réalise ces opérations avec les données d'entrainements de notre projet :

In [16]:
MAX_NB_MOTS = 1000000
MAX_SEQUENCE_LENGTH = 1000

# Chargement des commentaires et des ressentis
commentaires = DataEntrainement['review'].astype(str).tolist()      # Récupère tous les commentaires dans une liste python
ressentis = DataEntrainement['polarity'].tolist()                   # Récupère tous les ressentis dans une liste python
labels = np.asarray(ressentis)                                      # Créé un tableau de type numpy avec les ressentis

# Encodage des commentaires
tokenizer = Tokenizer(num_words=MAX_NB_MOTS)                              # Initialise la fonction Tokenizer de Keras
tokenizer.fit_on_texts(commentaires)                                      # Création des index des mots
sequences = tokenizer.texts_to_sequences(commentaires)                    # Transformation des phrases en séquences d'index de mots 
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)   # Bourrage des vecteurs

In [ ]:
padded_sequences

In [ ]:
tokenizer.index_word

**Création des données d'entrainement et de tests**  
On utilise la fonction `train_test_split` de [ScikitLearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) afin de créer les données d'entrainement et de tests à partir des séquences :




In [7]:
index_des_mots = tokenizer.word_index
nbr_mots = min(MAX_NB_MOTS, len(index_des_mots)) + 1

x_entrainement, x_test, y_entrainement, y_test = train_test_split(padded_sequences, labels, test_size=0.2)

In [ ]:
y_entrainement

**Création de la matrice embarquant les données numériques des vecteurs des mots contenus dans les commentaires**  
L'objectif est ici de créer une matrice dont chaque ligne contient le vecteur du mot issu de l'algorithme GolVe. La matrice est donc de taille n x m avec :  
* n : Nombre de mots (uniques) pris en compte dans l'ensemble des commentiares  
* m : Nombre de valeurs contenues dans les vecteurs GolvE (300 dans notre exemple)  
  
$$\begin{array}{*{20}{c}}
{glace}\\
{soda}\\
{...}\\
{film}
\end{array}\left( \begin{array}{l}
\begin{array}{*{20}{c}}
{ - 0.3}&{0.2}&{...}&{0.32}&{ - 0.24}
\end{array}\\
\begin{array}{*{20}{c}}
{ - 0.1}&{0.3}&{...}&{0.52}&{ - 0.94}
\end{array}\\
\begin{array}{*{20}{c}}
{...}&{...}&{...}&{...}&{...}
\end{array}\\
\begin{array}{*{20}{c}}
{ - 0.5}&{0.9}&{...}&{0.72}&{ - 0.24}
\end{array}
\end{array} \right)$$    
  
    

Les lignes ne sont pas arrangées dans n'importe quel ordre : Elles suivent l'ordre des séquences créées par la fonction `tokenizer.texts_to_sequences()`





Pour cela définit tout d'abord la fonction `Chargement_Vecteurs()` qui va créer un tableau de type numpy, avec pour chaque mot son vecteur correspondant :  


```
  ...
  'embêtant': array([-2.26152748e-01,  3.20324749e-01, -1.10406213e-01, -6.05279326e-01,
        -4.68072683e-01,  1.29561171e-01,  5.62916815e-01, -1.16834176e+00,
       ...
        -7.50736117e-01, -2.48611599e-01, -2.42264550e-02, -9.54209745e-01],
       dtype=float32),
 'lockheed': array([ 3.6074609e-01, -8.0667698e-01,  8.7549436e-01,  6.2351477e-01,
        -9.2155904e-01,  7.3180795e-01, -2.8121206e-01,  2.9078028e-01,
        ...
         1.5100185e+00,  8.1941241e-01, -1.6970781e+00,  1.9289741e-01],
       dtype=float32),
 'séparez': array([-0.5703459 , -0.8884122 , -0.4579496 ,  0.55588883, -0.8727098 ,
         0.56783265, -0.10067926,  0.14027229, -0.89301944, -0.42706665,
        ...
        -0.36254016, -0.40695533,  1.087127  , -0.641696  ,  0.10919298,
        ...
```  

Puis la fonction `Creation_Matrice()` qui va créer notre matrice. Pour chaque mot contenu dans les séquences créées par la fonction `tokenizer.texts_to_sequences` on récupère le vecteur correspondant à l'aide du tableau numpy créé précédemment. Ainsi chaque ligne de la matrice indexe chaque mot des séquences.


In [17]:
MAX_NB_VECTORS = 400000
EMBEDDING_DIM = 300

def Chargement_Vecteurs():
    print('Chargement des vecteurs GloVe...')
    glove_dict = {}
    Max_Nb_Vect = 0
    with open('/content/data/multilingual_embeddings.fr', encoding='utf8') as fichier:
        for ligne in fichier:
            Max_Nb_Vect = Max_Nb_Vect + 1
            if Max_Nb_Vect > MAX_NB_VECTORS:
              break
            valeur = ligne.split()
            mot = valeur[0]
            glove_dict[mot] = np.asarray(valeur[1:], dtype='float32')
    return glove_dict

def Creation_Matrice(index_mot, nbr_mots):
    glove_dict = Chargement_Vecteurs()
    matrice = np.zeros((nbr_mots, EMBEDDING_DIM))
    for mot, i in index_mot.items():
        if i > nbr_mots:
            continue
        vector = glove_dict.get(mot)
        if vector is not None:
            matrice[i] = vector
    print('Matrice créée...')
    return matrice

Regardons à quoi cela ressemble sur notre précédent exemple :

In [ ]:
print(com)
print(tokenizer_ex.word_index)

matrice = Creation_Matrice(tokenizer_ex.word_index, 10)

In [ ]:
matrice

On créé donc maintenant la matrice pour notre projet :

In [ ]:
matrice = Creation_Matrice(index_des_mots, nbr_mots)

In [ ]:
matrice.shape

# Définition du Modèle

Nous utilisons un réseau de neurones à convolution 1D avec Keras et en utilisant la matrice créée précédemment afin de paramétrer la couche interne. La couche interne est pré-entrainée donc nous n'avons pas d'entrainement à réaliser sur celle-ci.  

La structure du réseau est la suivante :  

In [ ]:
from IPython.display import Image
Image(url='https://github.com/AlexandreBourrieau/ML-F1/blob/master/Carnets%20Jupyter/Images/Conv1D2.png?raw=1', width=1500)

In [ ]:
dropout = 0.4

model = Sequential()
model.add(Embedding(nbr_mots, EMBEDDING_DIM, weights=[matrice],
                    input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model.add(Dropout(dropout))

model.add(Conv1D(128, 5, activation='relu', padding='same', strides=2))
model.add(GlobalMaxPooling1D())
model.add(Dropout(dropout))

model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(2, activation='softmax'))
model.summary()

# Entrainement du modèle

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['acc'])

# Entraine le modèle sur un certain nombre d'itérations
historique = model.fit(x_entrainement, y_entrainement, batch_size=128, epochs=40, verbose=1, validation_data=(x_test, y_test))

# Evalue le modèle avec les échantillons de tests
score = model.evaluate(x_test, y_test, verbose=0)
print('Fonction d\'objectif des tests :', score[0])
print('Précision des tests :', score[1])

# Synthèse
plt.plot(historique.history['acc'])
plt.plot(historique.history['val_acc'])
plt.title('Précision du modèle')
plt.ylabel('Précision')
plt.xlabel('Itérations')
plt.legend(['Entrainements', 'Tests'], loc='upper left')
plt.show()

# Prédictions



In [21]:
predictions = model.predict(padded_sequences)
plus_probable = predictions.argmax(1)

In [ ]:
index = random.randrange(len(predictions))
print(commentaires[index])
print('Prédiction: %d, label: %d' % (plus_probable[index], ressentis[index]))

# Analyse des erreurs

In [ ]:
for i in range(10000):
    index = random.randrange(len(predictions))
    if plus_probable[index] != ressentis[index]:
        break

print(commentaires[index])
print('Prédiction: %d, label: %d' % (plus_probable[index], ressentis[index]))

plt.bar(range(2), predictions[index], tick_label=range(2))
plt.title('Valeur prédite ')
plt.show()